
scheduler index?

ROUTE = /schedule

ROOT = true

METHOD = get


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Schedule a Meeting</title>
  <link href="https://cdn.jsdelivr.net/npm/fullcalendar@3.3.0/dist/fullcalendar.min.css" rel="stylesheet" />
  <link rel="stylesheet" href="/schedule-style.css" />
</head>

<body>

  <div id="calendar"></div>

  <!-- Modal for meeting details -->
  <div id="meetingModal" class="modal">
    <div class="modal-content">
      <h2>Schedule a Meeting</h2>
      <form id="meetingForm">
        <label for="summary">Meeting Summary:</label>
        <input type="text" id="summary" name="summary" required><br>

        <label for="location">Location:</label>
        <input type="text" id="location" name="location" required><br>

        <label for="description">Description:</label>
        <textarea id="description" name="description" required></textarea><br>

        <label for="startDateTime">Start Date and Time:</label>
        <input type="datetime-local" id="startDateTime" name="startDateTime" required><br>

        <label for="endDateTime">End Date and Time:</label>
        <input type="datetime-local" id="endDateTime" name="endDateTime" required><br>

        <label for="email">Your Email:</label>
        <input type="email" id="email" name="email" required><br>

        <button type="submit">Schedule Meeting</button>
      </form>
    </div>
  </div>

  <script src="https://cdnjs.cloudflare.com/ajax/libs/moment.js/2.29.4/moment.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/moment-timezone/0.5.43/moment-timezone-with-data.min.js"></script>
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/fullcalendar@3.3.0/dist/fullcalendar.min.js"></script>
  <script src="/schedule-script.js"></script>

</body>

</html>

schedule stylesheet?

ROUTE = /schedule-style.css


In [ ]:

* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

body {
  font-family: Arial, sans-serif;
  display: flex;
  justify-content: center;
  align-items: center;
  min-height: 100vh;
  margin: 0;
  background-color: #f4f4f4;
}

#calendar {
  width: 80%;
  margin: 50px auto;
}

.modal {
  display: none;
  position: fixed;
  z-index: 1;
  left: 0;
  top: 0;
  width: 100%;
  height: 100%;
  overflow: auto;
  background-color: rgb(0,0,0);
  background-color: rgba(0,0,0,0.4);
}

.modal-content {
  background-color: #fff;
  margin: 15% auto;
  padding: 20px;
  border: 1px solid #888;
  width: 300px;
  text-align: center;
}

label {
  display: block;
  margin-top: 10px;
}

input {
  width: 100%;
  padding: 8px;
  margin-top: 5px;
}

button {
  padding: 10px 20px;
  background-color: #4CAF50;
  color: white;
  border: none;
  cursor: pointer;
}

button:hover {
  background-color: #45a049;
}



schedule client script?

ROUTE = /schedule-script.js


In [ ]:
$(document).ready(function() {
  // Initialize FullCalendar
  $('#calendar').fullCalendar({
  events: [], // Add your events here if needed
  dayClick: function(date, jsEvent, view) {

    // Open the modal
    $('#meetingModal').show();
  
    const start = new moment(date.toISOString()).startOf('day').format('YYYY-MM-DDTHH:mm')
    const end = new moment(date.toISOString()).endOf('day').format('YYYY-MM-DDTHH:mm')

    // Set start and end datetime fields correctly without mutating the original
    $('#startDateTime').val(start);
    $('#endDateTime').val(end);

    //$('#meetingForm').data('start', start);
    //$('#meetingForm').data('end', end);

  }
});
  
  $(document).on('click', '#meetingModal', function(e) {
    if (!$(e.target).closest('.modal-content').length) {
      $('#meetingModal').hide(); // or use fadeOut()
    }
  });

  // Handle form submission
  $('#meetingForm').submit(function(event) {
    event.preventDefault(); // Prevent the default form submission

    // Gather form data
    var formData = {
      email: $('#email').val(),
      summary: $('#summary').val(),
      location: $('#location').val(),
      description: $('#description').val(),
      startDateTime: $('#startDateTime').val(),
      clientTimezone: moment.tz.guess(),
      endDateTime: $('#endDateTime').val(),
    };

    // Send the data via POST request
    $.ajax({
      url: '/schedule', // Your backend endpoint
      method: 'POST',
      contentType: 'application/json',
      data: JSON.stringify(formData),
      success: function(response) {
        // Handle successful response
        alert('Meeting scheduled successfully!');
        $('#meetingModal').hide(); // Close the modal
        $('#meetingForm')[0].reset(); // Reset the form
      },
      error: function(error) {
        // Handle error
        alert('Failed to schedule the meeting. Please try again.');
        console.error('Error:', error);
      }
    });
  });
});



schedule service?

ROUTE = /schedule

METHOD = post


In [ ]:
const moment = require('moment')
const authorizeCalendar = importer.import('authorize google calendar'); // Authorization function

async function createEvent(email, summary, location, description, startDateTime, endDateTime, clientTimezone) {
  let calendar;
  try {
    // Authorize and get the calendar service
    calendar = await authorizeCalendar();
    
    // Prepare event object
    const event = {
      summary: summary,
      location: location,
      description: description,
      start: {
        dateTime: moment(startDateTime).format('YYYY-MM-DDTHH:mm:ss'),
        timeZone: clientTimezone
      },
      end: {
        dateTime: moment(endDateTime).format('YYYY-MM-DDTHH:mm:ss'),
        timeZone: clientTimezone
      },
      attendees: [
        { email: email }
      ],
      reminders: {
        useDefault: true
      },
    };

    // Insert event into the calendar
    const response = await calendar.events.insert({
      calendarId: 'primary', // Can be customized to other calendars
      resource: event,
    });

    console.log('Event created: ', response.data);
    return response.data;
  } catch (error) {
    console.error('Error creating event: ', error);
    throw error;
  }
}

module.exports = createEvent
